In [1]:
def preprocess_string(s):
    plaintext = ''
    for i in s:
        temp = ord(i)
        if (temp < 123 and temp > 96 or temp < 91 and temp > 64):
            plaintext += i
    return plaintext

# 1.

In [2]:
def create_loc_data(key):
    loc = {}
    for i in range(len(key)):
        for j in range(len(key[i])):
            for l in key[i][j]:
                loc[l] = (i, j)
    return loc
            
def playfair(s, key, bogus = 'X'):
    s = s.upper()
    pairs = []
    i = 0
    l = len(s)
    while(i < l):
        if (s[i] == s[i + 1]):
            pairs.append(s[i] + bogus)
            i += 1
        else:
            pairs.append(s[i] + s[i + 1])
            i += 2
        if (i + 2 > l):
            s = s + bogus
    
    loc = create_loc_data(key)
    ciphertext = ''
    for i, j in pairs:
        
        loc_i = loc[i]
        loc_j = loc[j]
        
        if (loc_i[0] == loc_j[0]):
            ciphertext += key[loc_i[0]][(loc_i[1] + 1) % 5][0]
            ciphertext += key[loc_j[0]][(loc_j[1] + 1) % 5][0]
        
        elif (loc_i[1] == loc_j[1]):
            ciphertext += key[(loc_i[0] + 1) % 5][loc_j[1]][0]
            ciphertext += key[(loc_j[0] + 1) % 5][loc_j[1]][0]
        
        else:
            ciphertext += key[loc_i[0]][loc_j[1]][0]
            ciphertext += key[loc_j[0]][loc_i[1]][0]
    
    return ciphertext

In [3]:
key = [['L', 'G', 'D', 'B', 'A'],
       ['Q', 'M', 'H', 'E', 'C'],
       ['U', 'R', 'N', 'IJ', 'F'],
       ['X', 'V', 'S', 'O', 'K'],
       ['Z', 'Y', 'W', 'T', 'P']]

In [4]:
s = input("Input for playfair encoder: ")
plaintext = preprocess_string(s)
print(playfair(plaintext, key))

Input for playfair encoder: we are discovered, save yourself
THGFHBNOEKOMIMHWGKMTXINVQBUK


# 2.

In [5]:
def string_to_key(s, leave = 'J', dual_with = 'I'):
    s = s + 'abcdefghijklmnopqrstuvwxyz'
    s = s.upper()
    l = [0] * 26;
    key_string = ''
    for i in s:
        if (l[ord(i) - 65] or i == leave):
            continue;
        else:
            key_string += i
            l[ord(i) - 65] = 1
            
    key = []
    idx = 0
    for i in range(5):
        row = []
        for j in range(5):
            if (key_string[idx] == dual_with):
                row.append(dual_with + leave)
            else:
                row.append(key_string[idx])
            idx += 1
        key.append(row)
    
    return key

In [7]:
s = input("Input for creating key: ") 
leave = input("Which alphabet to leave out: ")
dual_with = input("With which missing alphabet forms pair: ")
key = string_to_key(s, leave, dual_with)
print(key)

Input for creating key: commonlounge
Which alphabet to leave out: P
With which missing alphabet forms pair: Q
[['C', 'O', 'M', 'N', 'L'], ['U', 'G', 'E', 'A', 'B'], ['D', 'F', 'H', 'I', 'J'], ['K', 'QP', 'R', 'S', 'T'], ['V', 'W', 'X', 'Y', 'Z']]


In [8]:
s = input("Input for playfair encoder: ")
plaintext = preprocess_string(s)
print(playfair(plaintext, key))

Input for playfair encoder: PL AY FA IR ME SS AG E
TOINIGHSEHRYYIEA


# 3.

In [10]:
def vigenere_cipher(plaintext, key):
    plaintext = plaintext.upper()
    key = key.upper()
    ciphertext = ''
    plaintext_len = len(plaintext)
    key_len = len(key)
    for i in range(0, plaintext_len, key_len):
        curr = plaintext[i:min(i + key_len, plaintext_len)]
        curr_encrypted = [chr((ord(curr[j]) + ord(key[j]) - 130) % 26 + 65) for j in range(len(curr))]
        curr_encrypted = ''.join(curr_encrypted)
        ciphertext += curr_encrypted
    return ciphertext

In [11]:
key = input("Enter key for encryption: ")
s = input("Input for vigenere cipher encoder: ")
plaintext = preprocess_string(s)
print(vigenere_cipher(plaintext, key))

Enter key for encryption: PASCAL
Input for vigenere cipher encoder: She is listening
HHWKSWXSLGNTCG


# 4.

#### Assuming Given Information : 'of the' occurs at least once in the text

In [12]:
import numpy as np
import math

def gcd(a, b):
    if (b == 0):
        return a;
    else:
        return gcd(b, a % b);

def extended_gcd(a, b):
    r1 = a
    r2 = b
    s1 = 1
    s2 = 0
    t1 = 0
    t2 = 1
    while(r2 > 0):
        q = r1 // r2

        r = r1 - q * r2
        r1 = r2
        r2 = r

        s = s1 - q * s2
        s1 = s2
        s2 = s

        t = t1 - q * t2
        t1 = t2
        t2 = t
    return [r1, s1, t1]

def inverse(n, b):
    if gcd(n, b) == 1:
        return [(n - b), extended_gcd(n, b)[2] % n]
    else:
        return [n - b, -1]
    
def get_matrix_inverse(a, n = 26):
    
    a = np.array(a, dtype = np.int32)
    n, m = a.shape
    if n != m:
        return -1
    else:
        a_inv = np.linalg.inv(a)
        a_det = round(np.linalg.det(a))
        a_det_inverse = inverse(26, a_det % 26)[1]
        if (a_det_inverse == -1):
            return -1
        a_inv = a_inv * a_det * a_det_inverse
        
    for i in range(n):
        for j in range(m):
            a_inv[i][j] = round(a_inv[i][j]) % 26
    a_inv = a_inv.astype(int)
            
    return a_inv

In [13]:
def hill_cryptoanalysis(s):
    
    occuring_pattern = 'ofthe'
    ciphertext_bunch = [s[i:i+4] for i in range(0, len(s)-3)]
    
    flag = True
    for i in ciphertext_bunch:
        p_int = [ord(j) - 97 for j in i]
        c_int = [ord(j) - 97 for j in occuring_pattern]
        p = np.array([[p_int[0], p_int[1]], [p_int[2], p_int[3]]])
        c = np.array([[c_int[0], c_int[1]], [c_int[2], c_int[3]]])
        p_inv = get_matrix_inverse(p)
        if (isinstance(p_inv, np.ndarray)):
            key = np.matmul(get_matrix_inverse(c), p) 

            n, m = key.shape
            for x in range(n):
                for y in range(m):
                    key[x][y] = key[x][y] % 26

            plaintext = ''
            ciphertext_bunches = [s[i:i+4] for i in range(0, len(s)-3, 4)]
            key_inverse = get_matrix_inverse(key)
            if (isinstance(key_inverse, np.ndarray)):
                for bunch in ciphertext_bunches:
                    bunch_int = [ord(c) - 97 for c in bunch]
                    bunch_matrix = np.array([[bunch_int[0], bunch_int[1]], [bunch_int[2], bunch_int[3]]])
                    plaintext_matrix = np.matmul(bunch_matrix, key_inverse)
                    for x in range(n):
                        for y in range(m):
                            plaintext += chr(plaintext_matrix[x][y] % 26 + 97)

            print('Key:', key, '\n', plaintext, '\n--------------------------------------------------\n')

            c = np.array([[c_int[1], c_int[2]], [c_int[3], c_int[4]]])
            key = np.matmul(get_matrix_inverse(c), p) 

            for x in range(n):
                for y in range(m):
                    key[x][y] = key[x][y] % 26

            plaintext = ''
            key_inverse = get_matrix_inverse(key)
            if (isinstance(key_inverse, np.ndarray)):
                for bunch in ciphertext_bunches:
                    bunch_int = [ord(c) - 97 for c in bunch]
                    bunch_matrix = np.array([[bunch_int[0], bunch_int[1]], [bunch_int[2], bunch_int[3]]])
                    plaintext_matrix = np.matmul(bunch_matrix, key_inverse)
                    for x in range(n):
                        for y in range(m):
                            plaintext += chr(plaintext_matrix[x][y] % 26 + 97)

            print('Key:', key, '\n', plaintext, '\n--------------------------------------------------\n')

In [14]:
s = input("Enter text: ")
s = preprocess_string(s)
hill_cryptoanalysis(s)

Enter text: 'fupcmtgzkyukbqfjhuktz kkixtta'"^8
Key: [[15 25]
 [13 24]] 
 wfofthdzsskquvjeklfbgfsafoqf 
--------------------------------------------------

Key: [[10 19]
 [ 5 17]] 
 frfthezyssqyvfetlvbafvaconfz 
--------------------------------------------------

Key: [[20  5]
 [17 10]] 
 cbulrwhmygycofthstbemvskbrwp 
--------------------------------------------------

Key: [[13  9]
 [24 19]] 
 brlzwhmxgicqfthetdevvhkiripx 
--------------------------------------------------

Key: [[ 5  0]
 [10  1]] 
 nuzcgtyzgykkpqjjduqtlkmidtja 
--------------------------------------------------

Key: [[ 9  1]
 [19  0]] 
 uzctttzjywkkqnjwuvtxkniutkab 
--------------------------------------------------

Key: [[ 0  5]
 [ 1 19]] 
 lflpfmdgykeupbofthxklzakkxlt 
--------------------------------------------------

Key: [[ 1 18]
 [ 0 19]] 
 fkpqmrgxkauybefthekxzwkgxxti 
--------------------------------------------------

Key: [[17 15]
 [11 14]] 
 ezehdnhleqsawllaofthepaituef 
---------------------------------